## Parse & Annotate

In [3]:
import numpy as np
import pandas as pd
import configparser
config = configparser.ConfigParser()

In [4]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [5]:
config.read("../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [6]:
!ls -l {data_home}

total 89664
-rw-r--r--@ 1 Samantha  staff   1752309 Jan 27 17:42 austen-persuasion.csv
drwxr-xr-x@ 9 Samantha  staff       288 Jan 16 08:52 gutenberg
-rw-r--r--@ 1 Samantha  staff  43681675 Feb 28 14:01 novels-CORPUS.csv
-rw-r--r--@ 1 Samantha  staff       421 Feb 28 14:00 novels-LIB.csv
-rw-r--r--@ 1 Samantha  staff    465627 Jan 20 15:27 pg42324.txt
drwxr-xr-x@ 6 Samantha  staff       192 Apr  7 20:49 woolf


In [7]:
data_directory = f"{data_home}/woolf"
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

#### Get lines

In [9]:
# The Voyage Out
voyage = f"{data_directory}/voyage.txt"
LINES_voyage = pd.DataFrame(open(voyage, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES_voyage.index.name = 'line_num'
LINES_voyage.line_str = LINES_voyage.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

# Night and Day
nightday = f"{data_directory}/nightday.txt"
LINES_nightday = pd.DataFrame(open(nightday, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES_nightday.index.name = 'line_num'
LINES_nightday.line_str = LINES_nightday.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

# Jacob's Room
jacob = f"{data_directory}/jacobsroom.txt"
LINES_jacob = pd.DataFrame(open(jacob, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES_jacob.index.name = 'line_num'
LINES_jacob.line_str = LINES_jacob.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

# Mrs. Dalloway
dalloway = f"{data_directory}/dalloway.txt"
LINES_dalloway = pd.DataFrame(open(dalloway, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES_dalloway.index.name = 'line_num'
LINES_dalloway.line_str = LINES_dalloway.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

#### Clean up

In [11]:
books = [LINES_voyage, LINES_nightday, LINES_jacob, LINES_dalloway]

def clip(book):
    clip_pats = [
        r"\*\*\*\s*START OF (?:THE|THIS) PROJECT",
        r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
    ]
    
    pat_a = book.line_str.str.match(clip_pats[0])
    pat_b = book.line_str.str.match(clip_pats[1])

    line_a = book.loc[pat_a].index[0] + 1
    line_b = book.loc[pat_b].index[0] - 1

    return book.loc[line_a : line_b]

LINES_voyage = clip(LINES_voyage)
LINES_nightday = clip(LINES_nightday)
LINES_jacob = clip(LINES_jacob)
LINES_dalloway = clip(LINES_dalloway)

#### Chunk by chapter (sections for Mrs. Dalloway)

voyage:

In [14]:
# First get rid of table of contents
LINES_voyage = LINES_voyage.iloc[38:]

chap_pat = r"^\s*CHAPTER\s+[IVXLCDM]+\s*$"
chap_lines_voyage = LINES_voyage.line_str.str.match(chap_pat, case=False)

In [15]:
LINES_voyage.loc[chap_lines_voyage, 'chap_num'] = [i+1 for i in range(LINES_voyage.loc[chap_lines_voyage].shape[0])]

In [16]:
LINES_voyage.chap_num = LINES_voyage.chap_num.ffill()

In [17]:
LINES_voyage = LINES_voyage.dropna(subset=['chap_num']) # Remove everything before Chapter 1
LINES_voyage = LINES_voyage.loc[~chap_lines_voyage] # Remove chapter heading lines; their work is done
LINES_voyage.chap_num = LINES_voyage.chap_num.astype('int') # Convert chap_num from float to int

In [18]:
LINES_voyage.sample(3)

,line_str,chap_num
line_num,,
3705,The present method seems to me abominable. Thi...,8
12031,"was torn across, and peace being established, ...",22
7049,"unusually dull, the women unusually badly dres...",14


In [19]:
# Make CHAPS table
CHAPS_voyage = LINES_voyage.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

CHAPS_voyage['chap_str'] = CHAPS_voyage.chap_str.str.strip()

In [20]:
CHAPS_voyage

,chap_str
chap_num,
1,As the streets that lead from the Strand to th...
2,"Uncomfortable as the night, with its rocking m..."
3,Early next morning there was a sound as of cha...
4,Next morning Clarissa was up before anyone els...
5,She was not able to follow up her observations...
6,“That’s the tragedy of life—as I always say!” ...
7,From a distance the _Euphrosyne_ looked very s...
8,"The next few months passed away, as many years..."
9,"An hour passed, and the downstairs rooms at th..."


nightday:

In [22]:
LINES_nightday = LINES_nightday.iloc[56:]

In [23]:
chap_pat = r"^\s*CHAPTER\s+[IVXLCDM]+\s*$"
chap_lines_nightday = LINES_nightday.line_str.str.match(chap_pat, case=False)

In [24]:
LINES_nightday.loc[chap_lines_nightday, 'chap_num'] = [i+1 for i in range(LINES_nightday.loc[chap_lines_nightday].shape[0])]

In [25]:
LINES_nightday.chap_num = LINES_nightday.chap_num.ffill()

In [26]:
LINES_nightday = LINES_nightday.dropna(subset=['chap_num']) # Remove everything before Chapter 1
LINES_nightday = LINES_nightday.loc[~chap_lines_nightday] # Remove chapter heading lines
LINES_nightday.chap_num = LINES_nightday.chap_num.astype('int') # Convert chap_num from float to int

In [27]:
LINES_nightday.sample(3)

,line_str,chap_num
line_num,,
6329,but he did not allow himself to yield to the p...,15
891,It was out of the question that she should put...,2
15791,,31


In [28]:
# Make CHAPS table
CHAPS_nightday = LINES_nightday.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

CHAPS_nightday['chap_str'] = CHAPS_nightday.chap_str.str.strip()

In [29]:
CHAPS_nightday

,chap_str
chap_num,
1,"It was a Sunday evening in October, and in com..."
2,The young man shut the door with a sharper sla...
3,Denham had accused Katharine Hilbery of belong...
4,"At about nine o’clock at night, on every alter..."
5,Denham had no conscious intention of following...
6,Of all the hours of an ordinary working week-d...
7,"“And little Augustus Pelham said to me, ‘It’s ..."
8,"She took her letters up to her room with her, ..."
9,Katharine disliked telling her mother about Cy...


jacob:

In [31]:
LINES_jacob = LINES_jacob.iloc[20:]

In [32]:
chap_pat = r"^\s*CHAPTER\s+[A-Z]+\s*$"
chap_lines_jacob = LINES_jacob.line_str.str.match(chap_pat, case=False)

In [33]:
LINES_jacob.loc[chap_lines_jacob, 'chap_num'] = [i+1 for i in range(LINES_jacob.loc[chap_lines_jacob].shape[0])]

In [34]:
LINES_jacob.chap_num = LINES_jacob.chap_num.ffill()

In [35]:
LINES_jacob = LINES_jacob.dropna(subset=['chap_num']) # Remove everything before Chapter 1
LINES_jacob = LINES_jacob.loc[~chap_lines_jacob] # Remove chapter heading lines
LINES_jacob.chap_num = LINES_jacob.chap_num.astype('int') # Convert chap_num from float to int

In [36]:
LINES_jacob.sample(3)

,line_str,chap_num
line_num,,
4145,"Fanny Elmer, believing implicitly in the truth...",10
2740,,6
5733,Square it stirs (since it is summer-time and t...,12


In [37]:
# Make CHAPS table
CHAPS_jacob = LINES_jacob.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

CHAPS_jacob['chap_str'] = CHAPS_jacob.chap_str.str.strip()

In [38]:
CHAPS_jacob

,chap_str
chap_num,
1,"""So of course,"" wrote Betty Flanders, pressing..."
2,"""MRS. FLANDERS""--""Poor Betty Flanders""--""Dear ..."
3,"""This is not a smoking-carriage,"" Mrs. Norman ..."
4,"What's the use of trying to read Shakespeare, ..."
5,"""I rather think,"" said Jacob, taking his pipe ..."
6,"The flames had fairly caught.\n\n""There's St. ..."
7,About this time a firm of merchants having dea...
8,"About half-past nine Jacob left the house, his..."
9,The Countess of Rocksbier sat at the head of t...


dalloway:

In [40]:
LINES_dalloway = LINES_dalloway.iloc[68:]

In [41]:
fake_break = pd.DataFrame({'line_str': ['* * * * *']})
LINES_dalloway = pd.concat([fake_break, LINES_dalloway], ignore_index=True)

LINES_dalloway.reset_index(drop=True, inplace=True)
LINES_dalloway.index.name = 'line_num'

In [42]:
chap_pat = r"^\s*(\*\s*){3,}\s*$" # mrs dalloway doesn't have chapters, but i am treating sections as chapters
chap_lines_dalloway = LINES_dalloway.line_str.str.match(chap_pat, case=False)

In [43]:
len(LINES_dalloway.loc[chap_lines_dalloway]) # 9 sections

9

In [44]:
LINES_dalloway.loc[chap_lines_dalloway, 'chap_num'] = [i+1 for i in range(LINES_dalloway.loc[chap_lines_dalloway].shape[0])]

In [45]:
LINES_dalloway.chap_num = LINES_dalloway.chap_num.ffill()

In [46]:
LINES_dalloway = LINES_dalloway.dropna(subset=['chap_num']) # Remove everything before Chapter 1
LINES_dalloway = LINES_dalloway.loc[~chap_lines_dalloway] # Remove chapter heading lines
LINES_dalloway.chap_num = LINES_dalloway.chap_num.astype('int') # Convert chap_num from float to int

In [47]:
LINES_dalloway.sample(3)

,line_str,chap_num
line_num,,
1321,"table, the mounted paper-knife, the dolphin an...",3
3401,"“I met Clarissa in the Park this morning,” sai...",7
5951,go back; the rooms were still crowded; people ...,8


In [48]:
# Make CHAPS table
CHAPS_dalloway = LINES_dalloway.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

CHAPS_dalloway['chap_str'] = CHAPS_dalloway.chap_str.str.strip()

In [49]:
# Get rid of "Transcriber's note" (chap_num 9)
CHAPS_dalloway = CHAPS_dalloway.drop(9)

In [50]:
CHAPS_dalloway

,chap_str
chap_num,
1,Mrs. Dalloway said she would buy the flowers h...
2,The violent explosion which made Mrs. Dalloway...
3,“What are they looking at?” said Clarissa Dall...
4,"Remember my party, remember my party, said Pet..."
5,The grey nurse resumed her knitting as Peter W...
6,So the elderly nurse knitted over the sleeping...
7,"It was awful, he cried, awful, awful!\n\nStill..."
8,"Lucy came running full tilt downstairs, having..."


#### Chunk by paragraph

In [52]:
para_pat = r'\n\n+'

In [53]:
def get_para(CHAPS, para_pat):
    OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']
    
    PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack().to_frame('para_str').sort_index()
    PARAS.index.names = OHCO[:2]

    PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
    PARAS['para_str'] = PARAS['para_str'].str.strip()
    PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

    return PARAS

In [54]:
PARAS_voyage = get_para(CHAPS_voyage, para_pat)
PARAS_voyage.head()

para_str
chap_num para_num                                                   
1        0         As the streets that lead from the Strand to th...
         1         One afternoon in the beginning of October when...
         2         The embankment juts out in angles here and the...
         3         Although Mrs. Ambrose stood quite still, much ...
         4         Lars Porsena of Clusium By the nine Gods he sw...

In [55]:
PARAS_nightday = get_para(CHAPS_nightday, para_pat)
PARAS_nightday.head()

para_str
chap_num para_num                                                   
1        0         It was a Sunday evening in October, and in com...
         1         Considering that the little party had been sea...
         2         That fact was perceptible to Mr. Denham also, ...
         3         “Now, what would you do if you were married to...
         4         “Surely she could learn Persian,” broke in a t...

In [56]:
PARAS_jacob = get_para(CHAPS_jacob, para_pat)
PARAS_jacob.head()

para_str
chap_num para_num                                                   
1        0         "So of course," wrote Betty Flanders, pressing...
         1         Slowly welling from the point of her gold nib,...
         2              "... nothing for it but to leave," she read.
         3         "Well, if Jacob doesn't want to play" (the sha...
         4         "Where IS that tiresome little boy?" she said....

In [57]:
PARAS_dalloway = get_para(CHAPS_dalloway, para_pat)
PARAS_dalloway.head()

para_str
chap_num para_num                                                   
1        0         Mrs. Dalloway said she would buy the flowers h...
         1         For Lucy had her work cut out for her. The doo...
         2         What a lark! What a plunge! For so it had alwa...
         3         She stiffened a little on the kerb, waiting fo...
         4         For having lived in Westminster--how many year...

#### Chunk by sentence

In [59]:
import re

def mask_abbreviations(text):
    # Replace known abbreviations with safe versions (no periods)
    abbr_map = {
        'Mr.': 'Mr',
        'Mrs.': 'Mrs',
        'Ms.': 'Ms',
        'Dr.': 'Dr',
        'Prof.': 'Prof',
        'St.': 'St',
        'Jr.': 'Jr',
        'Sr.': 'Sr',
        'etc.': 'etc',
        'e.g.': 'eg',
        'i.e.': 'ie',
    }
    for k, v in abbr_map.items():
        text = text.replace(k, v)
    return text

def get_sent(PARAS):
    OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

    # Mask abbreviations to avoid splitting on them
    PARA_MASKED = PARAS['para_str'].apply(mask_abbreviations)

    # Now split on punctuation followed by whitespace
    SENTS = PARA_MASKED.str.split(r'(?<=[.?!])\s+', expand=True).stack().to_frame('sent_str')
    SENTS.index.names = OHCO[:3]

    # Clean up
    SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')]
    SENTS['sent_str'] = SENTS['sent_str'].str.strip()

    return SENTS

In [60]:
SENTS_voyage = get_sent(PARAS_voyage)
SENTS_voyage.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         As the streets that lead from the Strand to th...
                  1         If you persist, lawyers’ clerks will have to m...
                  2         In the streets of London where beauty goes unr...
         1        0         One afternoon in the beginning of October when...
                  1                    Angry glances struck upon their backs.

In [61]:
SENTS_nightday = get_sent(PARAS_nightday)
SENTS_nightday.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         It was a Sunday evening in October, and in com...
                  1         Perhaps a fifth part of her mind was thus occu...
                  2         But although she was silent, she was evidently...
                  3         A single glance was enough to show that Mrs Hi...
         1        0         Considering that the little party had been sea...

In [62]:
SENTS_jacob = get_sent(PARAS_jacob)
SENTS_jacob.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         "So of course," wrote Betty Flanders, pressing...
         1        0         Slowly welling from the point of her gold nib,...
                  1         The entire bay quivered; the lighthouse wobble...
                  2                                       She winked quickly.
                  3                              Accidents were awful things.

In [63]:
SENTS_dalloway = get_sent(PARAS_dalloway)
SENTS_dalloway.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         Mrs Dalloway said she would buy the flowers he...
         1        0                    For Lucy had her work cut out for her.
                  1         The doors would be taken off their hinges; Rum...
                  2         And then, thought Clarissa Dalloway, what a mo...
         2        0                                              What a lark!

#### Chunk by token (get corpus tables!)

In [65]:
def get_tokens(SENTS):
    OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

    token_pat = r"[\s',-]+"
    TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack().to_frame('token_str')

    TOKENS.index.names = OHCO[:4]

    return TOKENS

In [66]:
TOKEN_voyage = get_tokens(SENTS_voyage)
TOKEN_voyage.head()

token_str
chap_num para_num sent_num token_num          
1        0        0        0                As
                           1               the
                           2           streets
                           3              that
                           4              lead

In [67]:
TOKEN_nightday = get_tokens(SENTS_nightday)
TOKEN_nightday.head()

token_str
chap_num para_num sent_num token_num          
1        0        0        0                It
                           1               was
                           2                 a
                           3            Sunday
                           4           evening

In [68]:
TOKEN_jacob = get_tokens(SENTS_jacob)
TOKEN_jacob.head()

token_str
chap_num para_num sent_num token_num          
1        0        0        0               "So
                           1                of
                           2            course
                           3                 "
                           4             wrote

In [69]:
TOKEN_dalloway = get_tokens(SENTS_dalloway)
TOKEN_dalloway.head()

token_str
chap_num para_num sent_num token_num          
1        0        0        0               Mrs
                           1          Dalloway
                           2              said
                           3               she
                           4             would

In [70]:
len(TOKEN_dalloway) + len(TOKEN_jacob) + len(TOKEN_nightday) + len(TOKEN_voyage)

427340

In [71]:
# Combine into one TOKEN (corpus) table

TOKEN_voyage['book_id'] = 1
TOKEN_nightday['book_id'] = 2
TOKEN_jacob['book_id'] = 3
TOKEN_dalloway['book_id'] = 4

In [72]:
TOKEN_voyage = TOKEN_voyage.set_index("book_id", append=True)
TOKEN_voyage = TOKEN_voyage.reorder_levels(["book_id", "chap_num", "para_num", "sent_num", "token_num"])

In [73]:
TOKEN_nightday = TOKEN_nightday.set_index("book_id", append=True)
TOKEN_nightday = TOKEN_nightday.reorder_levels(["book_id", "chap_num", "para_num", "sent_num", "token_num"])

In [74]:
TOKEN_jacob = TOKEN_jacob.set_index("book_id", append=True)
TOKEN_jacob = TOKEN_jacob.reorder_levels(["book_id", "chap_num", "para_num", "sent_num", "token_num"])

In [75]:
TOKEN_dalloway = TOKEN_dalloway.set_index("book_id", append=True)
TOKEN_dalloway = TOKEN_dalloway.reorder_levels(["book_id", "chap_num", "para_num", "sent_num", "token_num"])

In [76]:
TOKEN = pd.concat([TOKEN_voyage,TOKEN_nightday,TOKEN_jacob,TOKEN_dalloway])
TOKEN.sample(3)

token_str
book_id chap_num para_num sent_num token_num          
2       14       0        3        3               the
1       15       101      0        5             began
        2        8        2        2              good

In [77]:
TOKEN['term_str'] = TOKEN.token_str.replace(r'[\W_]+', '', regex=True).str.lower()

In [78]:
TOKEN.sample(3)

token_str term_str
book_id chap_num para_num sent_num token_num                   
3       5        9        1        43          pausing  pausing
2       33       58       2        18             into     into
        18       118      4        2                he       he

In [79]:
# Add pos and pos_group

import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('universal_tagset')

def pos_tag_sentence(group):
    tokens = group['token_str'].tolist()
    tags = pos_tag(tokens)
    group['pos'] = [tag for _, tag in tags]
    return group

TOKEN = TOKEN.groupby(['book_id', 'chap_num', 'para_num', 'sent_num']).apply(pos_tag_sentence)

def coarse_pos(tag):
    if tag.startswith('NN'):
        return 'NN'
    elif tag.startswith('VB'):
        return 'VB'
    elif tag.startswith('JJ'):
        return 'JJ'
    elif tag.startswith('RB'):
        return 'RB'
    elif tag.startswith('PR'):
        return 'PR'
    elif tag.startswith('WP'):
        return 'WP'
    elif tag in ['.', ',', ':', '(', ')', "''", '``']:
        return 'PUNCT'
    else:
        return tag

TOKEN['pos_group'] = TOKEN['pos'].apply(coarse_pos)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Samantha/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/Samantha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/Samantha/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
/var/folders/13/9xyr4p8x2kq9msv1m8ysrjq00000gn/T/ipykernel_14211/1295028410.py:18: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  TOKEN = TOKEN.groupby(['book_id', 'chap_n

In [80]:
TOKEN.head()

token_str term_str  pos pos_group
book_id chap_num para_num sent_num token_num                                  
1       1        0        0        0                As       as   IN        IN
                                   1               the      the   DT        DT
                                   2           streets  streets  NNS        NN
                                   3              that     that  WDT       WDT
                                   4              lead     lead  VBP        VB

#### Get VOCAB table from TOKEN table

In [82]:
VOCAB = TOKEN.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)

In [83]:
VOCAB.sample(5)

,n,n_chars,p,i
term_str,,,,
colonel,1,7,0.000002,18.705025
defrauded,1,9,0.000002,18.705025
sailors,10,7,0.000023,15.383097
recall,14,6,0.000033,14.897670
consulted,7,9,0.000016,15.897670


In [84]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [85]:
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [86]:
VOCAB[VOCAB.stop == 1].sample(3)

,n,n_chars,p,i,stop
term_str,,,,,
through,392,7,0.000917,10.090315,1
but,2721,3,0.006367,7.295104,1
above,128,5,0.000300,11.705025,1


In [87]:
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

In [88]:
VOCAB['max_pos'] = TOKEN[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = TOKEN[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)

In [89]:
VOCAB['n_pos_group'] = TOKEN[['term_str','pos_group']].value_counts().unstack().count(1)
VOCAB['cat_pos_group'] = TOKEN[['term_str','pos_group']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos_group.apply(lambda x: set(x))

In [90]:
VOCAB['n_pos'] = TOKEN[['term_str','pos']].value_counts().unstack().count(1)
VOCAB['cat_pos'] = TOKEN[['term_str','pos']].value_counts().to_frame('n').reset_index()\
    .groupby('term_str').pos.apply(lambda x: set(x))

In [91]:
VOCAB = VOCAB[1:]

In [92]:
VOCAB.sample(3).T

term_str,siberia,cranny,clevernot
n,1,1,1
n_chars,7,6,9
p,0.000002,0.000002,0.000002
i,18.705025,18.705025,18.705025
stop,0,0,0
stem_porter,siberia,cranni,clevernot
max_pos,NNP,NN,VB
max_pos_group,NN,NN,VB
n_pos_group,1,1,1
cat_pos_group,{NN},{NN},{VB}


In [93]:
# Compute df and idf to get VOCAB['dfidf']

## To do this I have to first create the BOW table, and then the DTCM, then compute DF and IDF.
## I'm going to use chapter as the bag. 

In [94]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']
bags = dict(
    SENTS = OHCO[:4],
    PARAS = OHCO[:3],
    CHAPS = OHCO[:2],
    BOOKS = OHCO[:1]
)
bag = 'CHAPS'

In [95]:
BOW = TOKEN.groupby(bags[bag]+['term_str']).term_str.count().to_frame('n')

In [96]:
DTCM = BOW.n.unstack(fill_value=0)

In [97]:
DF = DTCM.astype('bool').sum()

In [98]:
N = DTCM.shape[0]
idf = {
    'standard': np.log2(N / DF),
    'max': np.log2(DF.max() / DF),
    'smooth': np.log2((1 + N) / (1 + DF)) + 1
}
idf_method = 'standard'
IDF = idf[idf_method]

In [99]:
VOCAB['df'] = DF
VOCAB['idf'] = IDF

In [100]:
VOCAB['dfidf'] = VOCAB.df * VOCAB.idf

In [101]:
VOCAB.sample(3).T

term_str,milady,norfolk,led
n,2,10,67
n_chars,6,7,3
p,0.000005,0.000023,0.000157
i,17.705025,15.383097,12.638936
stop,0,0,0
stem_porter,miladi,norfolk,led
max_pos,JJ,NNP,VBD
max_pos_group,JJ,NN,VB
n_pos_group,1,1,1
cat_pos_group,{JJ},{NN},{VB}


#### Make LIB table

In [103]:
source_file_list = ['/Users/Samantha/Desktop/MSDS/DS5001/data/woolf/voyage.txt', # book_id 1
                    '/Users/Samantha/Desktop/MSDS/DS5001/data/woolf/nightday.txt', # 2
                    '/Users/Samantha/Desktop/MSDS/DS5001/data/woolf/jacobsroom.txt', # 3
                    '/Users/Samantha/Desktop/MSDS/DS5001/data/woolf/dalloway.txt' # 4
]

In [104]:
book_data = []
book_id = 1
for source_file_path in source_file_list:
    if book_id == 1:
        book_title = 'THE VOYAGE OUT'
    elif book_id == 2:
        book_title = 'NIGHT AND DAY'
    elif book_id == 3:
        book_title = 'JACOBS ROOM'
    elif book_id == 4:
        book_title = 'MRS DALLOWAY'
        
    book_data.append((book_id, source_file_path, book_title))
    book_id += 1

In [105]:
book_data

[(1,
  '/Users/Samantha/Desktop/MSDS/DS5001/data/woolf/voyage.txt',
  'THE VOYAGE OUT'),
 (2,
  '/Users/Samantha/Desktop/MSDS/DS5001/data/woolf/nightday.txt',
  'NIGHT AND DAY'),
 (3,
  '/Users/Samantha/Desktop/MSDS/DS5001/data/woolf/jacobsroom.txt',
  'JACOBS ROOM'),
 (4,
  '/Users/Samantha/Desktop/MSDS/DS5001/data/woolf/dalloway.txt',
  'MRS DALLOWAY')]

In [106]:
LIB = pd.DataFrame(book_data, columns=['book_id','source_file_path','raw_title'])\
    .set_index('book_id').sort_index()

In [107]:
LIB['book_len'] = TOKEN.groupby('book_id').term_str.count() # number of words

In [108]:
LIB['n_chaps'] = TOKEN.reset_index()[['book_id','chap_num']]\
    .drop_duplicates()\
    .groupby('book_id').chap_num.count()

In [109]:
dates = [1915,
         1919,
         1922,
         1925]
LIB['date'] = dates

In [110]:
TOKEN['n_chars'] = TOKEN['token_str'].str.len()
chars_per_book = TOKEN.groupby('book_id')['n_chars'].sum()

In [111]:
chars_per_book

book_id
1    612493
2    760337
3    250699
4    287737
Name: n_chars, dtype: int64

In [112]:
n_chars = [612493,
          760337,
          250699,
          287737]

In [113]:
LIB['n_chars'] = n_chars

In [114]:
LIB

,source_file_path,raw_title,book_len,n_chaps,date,n_chars
book_id,,,,,,
1,/Users/Samantha/Desktop/MSDS/DS5001/data/woolf...,THE VOYAGE OUT,138107,27,1915,612493
2,/Users/Samantha/Desktop/MSDS/DS5001/data/woolf...,NIGHT AND DAY,168196,34,1919,760337
3,/Users/Samantha/Desktop/MSDS/DS5001/data/woolf...,JACOBS ROOM,56548,14,1922,250699
4,/Users/Samantha/Desktop/MSDS/DS5001/data/woolf...,MRS DALLOWAY,64489,8,1925,287737


In [115]:
ages = [33,37,40,43]
LIB['woolf_age'] = ages

In [116]:
sexes = ['f','f','m','f']
LIB['prot_sex'] = sexes

In [221]:
LIB

,source_file_path,raw_title,book_len,n_chaps,date,n_chars,woolf_age,prot_sex
book_id,,,,,,,,
1,/Users/Samantha/Desktop/MSDS/DS5001/data/woolf...,THE VOYAGE OUT,138107,27,1915,612493,33,f
2,/Users/Samantha/Desktop/MSDS/DS5001/data/woolf...,NIGHT AND DAY,168196,34,1919,760337,37,f
3,/Users/Samantha/Desktop/MSDS/DS5001/data/woolf...,JACOBS ROOM,56548,14,1922,250699,40,m
4,/Users/Samantha/Desktop/MSDS/DS5001/data/woolf...,MRS DALLOWAY,64489,8,1925,287737,43,f


### SAVE MAIN TABLES TO FILES

In [223]:
TOKEN.to_csv('woolf-CORPUS.csv')
VOCAB.to_csv('woolf-VOCAB.csv')
LIB.to_csv('woolf-LIB.csv')

#### Finish BOW, DTCM, TFIDF, TFIDF_L2

In [245]:
tf = {
    'sum': (DTCM.T / DTCM.T.sum()).T,
    'max': (DTCM.T / DTCM.T.max()).T,
    'log': (np.log2(1 + DTCM.T)).T,
    'raw':  DTCM,
    'double_norm': (DTCM.T / DTCM.T.max()).T,
    'binary': DTCM.T.astype('bool').astype('int').T
}

In [287]:
TF = tf['max']

In [289]:
TFIDF = TF * IDF

In [291]:
TFIDF.sample(3)

term_str                    112  1215counted  1580  1660  1697  1780  1852  \
book_id chap_num                                                             
4       4         0.000457  0.0          0.0   0.0   0.0   0.0   0.0   0.0   
        3         0.001498  0.0          0.0   0.0   0.0   0.0   0.0   0.0   
2       25        0.003331  0.0          0.0   0.0   0.0   0.0   0.0   0.0   

term_str          1853  1860  ...  περῶν  πολιοῦ  πολλὰ  πόντου  τοῦτο   τὰ  \
book_id chap_num              ...                                             
4       4          0.0   0.0  ...    0.0     0.0    0.0     0.0    0.0  0.0   
        3          0.0   0.0  ...    0.0     0.0    0.0     0.0    0.0  0.0   
2       25         0.0   0.0  ...    0.0     0.0    0.0     0.0    0.0  0.0   

term_str          χειμερίῳ  χωρεῖ   ἀν   ὑπ  
book_id chap_num                             
4       4              0.0    0.0  0.0  0.0  
        3              0.0    0.0  0.0  0.0  
2       25             0.0    0.0  0.0  0.0  

[3 rows x 18496 columns]

In [293]:
BOW['tf'] = TF.stack()
BOW['tfidf'] = TFIDF.stack()

In [295]:
BOW.sort_values('tfidf', ascending=False).head()

n        tf     tfidf
book_id chap_num term_str                          
3       7        salvin      13  0.154762  0.986613
        11       cruttendon  33  0.172775  0.928672
        14       bonamy       6  0.272727  0.920465
2       29       cassandra   57  0.333333  0.855895
                 katharine   97  0.567251  0.779993

In [306]:
DTCM.sample(3)

term_str              112  1215counted  1580  1660  1697  1780  1852  1853  \
book_id chap_num                                                             
3       10        18    0            0     0     0     0     0     0     0   
        1         22    0            0     0     0     0     0     0     0   
2       15        39    0            1     0     0     0     0     0     0   

term_str          1860  ...  περῶν  πολιοῦ  πολλὰ  πόντου  τοῦτο  τὰ  \
book_id chap_num        ...                                            
3       10           0  ...      0       0      0       0      0   0   
        1            0  ...      0       0      0       0      0   0   
2       15           0  ...      0       0      0       0      0   0   

term_str          χειμερίῳ  χωρεῖ  ἀν  ὑπ  
book_id chap_num                           
3       10               0      0   0   0  
        1                0      0   0   0  
2       15               0      0   0   0  

[3 rows x 18496 columns]

In [360]:
## Get L2 norm of TFIDF

# Compute the L2 norm for each document vector (row)
l2_norms = np.sqrt((TFIDF ** 2).sum(axis=1))

# Divide each row by its L2 norm
TFIDF_L2 = TFIDF.div(l2_norms, axis=0)

In [362]:
# Now reduce to top N terms with highest mean tfidf
# Note that I haven't filtered out proper nouns so many top tfidf terms are names
N_terms = 1000

# Compute average TFIDF across documents for each term
mean_tfidf = TFIDF_L2.mean(axis=0)

# Select top N terms
top_terms = mean_tfidf.sort_values(ascending=False).head(N_terms).index

# Reduce the TFIDF_L2 matrix
TFIDF_L2 = TFIDF_L2[top_terms]

In [364]:
TFIDF_L2.T.sample(10)

book_id          1                                                    \
chap_num        1         2         3         4         5         6    
term_str                                                               
returned  0.000000  0.000000  0.000000  0.005189  0.000000  0.000000   
beauty    0.014813  0.010862  0.004315  0.004361  0.007504  0.000000   
figure    0.009184  0.000000  0.004013  0.004055  0.000000  0.006130   
past      0.012623  0.005554  0.000000  0.006689  0.000000  0.000000   
globe     0.000000  0.000000  0.026297  0.000000  0.000000  0.000000   
whom      0.003943  0.000000  0.003446  0.003482  0.000000  0.000000   
effort    0.000000  0.000000  0.000000  0.005016  0.008631  0.000000   
aunt      0.023752  0.078377  0.006919  0.000000  0.000000  0.000000   
struck    0.015772  0.000000  0.006892  0.000000  0.005992  0.000000   
happy     0.000000  0.004001  0.000000  0.012851  0.000000  0.004857   

book_id                                           ...         3       \
chap_num        7         8         9         10  ...        13   14   
term_str                                          ...                  
returned  0.026493  0.000000  0.008055  0.006644  ...  0.000000  0.0   
beauty    0.007421  0.000000  0.006768  0.011166  ...  0.000000  0.0   
figure    0.000000  0.007558  0.009441  0.010384  ...  0.000000  0.0   
past      0.000000  0.004155  0.003461  0.002855  ...  0.011088  0.0   
globe     0.000000  0.000000  0.000000  0.000000  ...  0.013217  0.0   
whom      0.005926  0.006490  0.002703  0.004459  ...  0.000000  0.0   
effort    0.000000  0.000000  0.000000  0.006422  ...  0.000000  0.0   
aunt      0.000000  0.000000  0.037985  0.000000  ...  0.000000  0.0   
struck    0.005926  0.000000  0.002703  0.000000  ...  0.000000  0.0   
happy     0.005467  0.000000  0.004987  0.000000  ...  0.003196  0.0   

book_id          4                                                    \
chap_num        1         2         3         4         5         6    
term_str                                                               
returned  0.000000  0.004651  0.000000  0.000000  0.000000  0.000000   
beauty    0.012158  0.019539  0.011632  0.000000  0.000000  0.000000   
figure    0.005653  0.003634  0.003606  0.000000  0.052581  0.000000   
past      0.003108  0.003996  0.005947  0.031277  0.007227  0.012728   
globe     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
whom      0.000000  0.003121  0.000000  0.010856  0.011288  0.006626   
effort    0.000000  0.000000  0.004460  0.015635  0.000000  0.009544   
aunt      0.000000  0.000000  0.024869  0.000000  0.000000  0.039916   
struck    0.004855  0.003121  0.003096  0.010856  0.000000  0.000000   
happy     0.013437  0.005758  0.005713  0.005008  0.000000  0.006114   

book_id                       
chap_num        7         8   
term_str                      
returned  0.003505  0.000000  
beauty    0.015708  0.004131  
figure    0.004565  0.003841  
past      0.005020  0.010560  
globe     0.000000  0.000000  
whom      0.010976  0.011545  
effort    0.003388  0.004751  
aunt      0.006297  0.016558  
struck    0.008624  0.001649  
happy     0.010127  0.009130  

[10 rows x 83 columns]

### SAVE DERIVED TABLES TO FILES

In [375]:
BOW.to_csv(f'woolf-BOW-{bag}.csv')
DTCM.to_csv(f'woolf-DTCM-{bag}.csv')
TFIDF.to_csv(f'woolf-TFIDF-{bag}.csv')
TFIDF_L2.to_csv(f'woolf-TFIDF_L2-{bag}.csv')

#### Compute metrics/outputs for final project notebook

In [393]:
# 6.1 Average length of each document in characters
LIB.n_chars.mean()

477816.5

In [407]:
# 6.3 List the top 20 significant words in the corpus by DFIDF
VOCAB.sort_values('dfidf', ascending=False).head(20).index

Index(['pages', 'wants', 'considerable', 'god', 'happiness', 'save', 'pink',
       'single', 'john', 'bedroom', 'agree', 'gentlemen', 'interrupted',
       'burst', 'explained', 'comes', 'compared', 'hat', 'anyhow', 'force'],
      dtype='object', name='term_str')

In [416]:
# 7.1 Number of observations
len(BOW)

113953